## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambamarca,PE,-6.6833,-78.5333,60.57,81,94,4.36,light rain
1,1,Cidreira,BR,-30.1811,-50.2056,74.91,89,100,3.87,overcast clouds
2,2,Kamaishi,JP,39.2667,141.8833,26.31,69,46,9.10,scattered clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,60.91,59,75,21.85,broken clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,80.19,84,100,19.01,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter your preferred minimum temperature for your trip"))
max_temp = float(input("Please enter your preferred maximum temperature for your trip"))

Please enter your preferred minimum temperature for your trip75
Please enter your preferred maximum temperature for your trip90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp)&(city_data_df["Max Temp"]>=min_temp)]
preferred_city_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Saint-Philippe,RE,-21.3585,55.7679,80.19,84,100,19.01,light rain
8,8,Hilo,US,19.7297,-155.0900,75.79,80,100,5.75,overcast clouds
11,11,Sabang,ID,5.8933,95.3214,80.83,75,96,11.16,overcast clouds
24,24,Rikitea,PF,-23.1203,-134.9692,79.09,73,96,10.54,overcast clouds
29,29,Cabo San Lucas,MX,22.8909,-109.9124,75.51,62,20,5.01,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_city_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185 entries, 4 to 713
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              185 non-null    int64  
 1   City                 185 non-null    object 
 2   Country              184 non-null    object 
 3   Lat                  185 non-null    float64
 4   Lng                  185 non-null    float64
 5   Max Temp             185 non-null    float64
 6   Humidity             185 non-null    int64  
 7   Cloudiness           185 non-null    int64  
 8   Wind Speed           185 non-null    float64
 9   Current Description  185 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 15.9+ KB


In [6]:
preferred_city_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_city_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

import numpy as np
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Saint-Philippe,RE,80.19,light rain,-21.3585,55.7679,NaN
8,Hilo,US,75.79,overcast clouds,19.7297,-155.0900,NaN
11,Sabang,ID,80.83,overcast clouds,5.8933,95.3214,NaN
24,Rikitea,PF,79.09,overcast clouds,-23.1203,-134.9692,NaN
29,Cabo San Lucas,MX,75.51,few clouds,22.8909,-109.9124,NaN
38,Vaini,TO,76.87,heavy intensity rain,-21.2000,-175.2000,NaN
39,La Romana,DO,79.16,scattered clouds,18.4273,-68.9728,NaN
41,Hithadhoo,MV,81.77,broken clouds,-0.6000,73.0833,NaN
44,Souillac,MU,79.36,light rain,-20.5167,57.5167,NaN
47,Carutapera,BR,75.49,overcast clouds,-1.1950,-46.0200,NaN


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel Not Found")
        

Hotel Not Found
Hotel Not Found
Hotel Not Found
Hotel Not Found
Hotel Not Found
Hotel Not Found
Hotel Not Found
Hotel Not Found
Hotel Not Found
Hotel Not Found


In [10]:
# 7. Drop the rows if there is no Hotel Name.
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184 entries, 4 to 713
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 184 non-null    object 
 1   Country              184 non-null    object 
 2   Max Temp             184 non-null    float64
 3   Current Description  184 non-null    object 
 4   Lat                  184 non-null    float64
 5   Lng                  184 non-null    float64
 6   Hotel Name           174 non-null    object 
dtypes: float64(3), object(4)
memory usage: 15.6+ KB


In [11]:
clean_hotel_df = hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0,30.1),zoom_level = 1.7)
heatlayer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,max_intensity=200, point_radius=4)
fig.add_layer(heatlayer)
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))